## 1. Setup and Data Loading

In [ ]:
# Essential Data Science Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Classification Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

# Class Imbalance Handling
try:
    from imblearn.over_sampling import SMOTE, RandomOverSampler
    from imblearn.under_sampling import RandomUnderSampler
    from imblearn.combine import SMOTETomek
    IMBALANCE_AVAILABLE = True
except ImportError:
    print("⚠️ imbalanced-learn not available. Will use basic resampling techniques.")
    IMBALANCE_AVAILABLE = False

# Evaluation Metrics
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score, f1_score,
    precision_score, recall_score, accuracy_score
)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"📊 NumPy version: {np.__version__}")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🤖 Imbalanced-learn available: {IMBALANCE_AVAILABLE}")

In [ ]:
# Data Loading Function
def load_fraud_data():
    """
    Load fraud detection datasets with error handling
    """
    try:
        # Try to load the actual datasets
        print("🔍 Looking for fraud detection datasets...")
        
        train_df = pd.read_csv('train_transaction.csv')
        test_df = pd.read_csv('test_transaction.csv')
        
        print(f"✅ Successfully loaded training data: {train_df.shape}")
        print(f"✅ Successfully loaded test data: {test_df.shape}")
        
        return train_df, test_df
        
    except FileNotFoundError:
        print("⚠️ Dataset files not found. Creating sample fraud detection dataset for demonstration...")
        
        # Create realistic sample fraud detection data
        np.random.seed(42)
        n_samples = 10000
        
        # Generate sample transaction data
        data = {
            'TransactionID': range(1, n_samples + 1),
            'TransactionAmt': np.random.lognormal(3, 1, n_samples),
            'ProductCD': np.random.choice(['W', 'C', 'R', 'H', 'S'], n_samples),
            'card1': np.random.randint(1000, 20000, n_samples),
            'card2': np.random.choice([100, 200, 300, 400, 500, None], n_samples),
            'card3': np.random.choice([100, 150, 185, None], n_samples),
            'card4': np.random.choice(['mastercard', 'visa', 'american express', 'discover'], n_samples),
            'card5': np.random.choice([100, 101, 102, 200, 201, None], n_samples),
            'card6': np.random.choice(['credit', 'debit'], n_samples),
            'addr1': np.random.randint(100, 600, n_samples),
            'addr2': np.random.randint(10, 100, n_samples),
            'dist1': np.random.exponential(50, n_samples),
            'dist2': np.random.exponential(50, n_samples),
            'P_emaildomain': np.random.choice(['gmail.com', 'yahoo.com', 'hotmail.com', None], n_samples),
            'R_emaildomain': np.random.choice(['gmail.com', 'yahoo.com', 'hotmail.com', None], n_samples),
            'C1': np.random.randint(0, 20, n_samples),
            'C2': np.random.randint(0, 20, n_samples),
            'C3': np.random.randint(0, 20, n_samples),
            'C4': np.random.randint(0, 50, n_samples),
            'C5': np.random.randint(0, 10, n_samples),
            'D1': np.random.exponential(10, n_samples),
            'D2': np.random.exponential(10, n_samples),
            'D3': np.random.exponential(10, n_samples),
            'V1': np.random.normal(0, 1, n_samples),
            'V2': np.random.normal(0, 1, n_samples),
            'V3': np.random.normal(0, 1, n_samples),
            'V4': np.random.normal(0, 1, n_samples),
            'V5': np.random.normal(0, 1, n_samples),
        }
        
        # Create fraud labels (imbalanced - about 3.5% fraud)
        fraud_probability = (
            0.01 +
            (data['TransactionAmt'] > 1000) * 0.05 +
            (pd.Series(data['card2']).isna()) * 0.02 +
            (pd.Series(data['P_emaildomain']).isna()) * 0.03 +
            np.random.normal(0, 0.01, n_samples)
        )
        
        fraud_probability = np.clip(fraud_probability, 0, 1)
        is_fraud = np.random.binomial(1, fraud_probability, n_samples)
        data['isFraud'] = is_fraud
        
        # Create DataFrames
        train_df = pd.DataFrame(data)
        
        # Create test set (without isFraud column)
        test_data = data.copy()
        del test_data['isFraud']
        test_data['TransactionID'] = range(n_samples + 1, 2 * n_samples + 1)
        test_df = pd.DataFrame(test_data)
        
        print(f"📊 Sample training data created: {train_df.shape}")
        print(f"📊 Sample test data created: {test_df.shape}")
        print(f"🎯 Fraud rate: {train_df['isFraud'].mean():.3f} ({train_df['isFraud'].sum()} fraudulent transactions)")
        
        return train_df, test_df

# Load the data
train_data, test_data = load_fraud_data()

In [ ]:
# Initial Data Inspection
print("=" * 60)
print("           DATASET OVERVIEW")
print("=" * 60)

print(f"\n📊 Training Data Shape: {train_data.shape}")
print(f"📊 Test Data Shape: {test_data.shape}")
print(f"🎯 Target Variable: isFraud")
print(f"🔍 Number of Features: {train_data.shape[1] - 1}")

# Check target variable
if 'isFraud' in train_data.columns:
    fraud_count = train_data['isFraud'].sum()
    total_count = len(train_data)
    fraud_rate = fraud_count / total_count
    
    print(f"\n🎯 TARGET VARIABLE ANALYSIS:")
    print(f"   • Total Transactions: {total_count:,}")
    print(f"   • Fraudulent Transactions: {fraud_count:,}")
    print(f"   • Legitimate Transactions: {total_count - fraud_count:,}")
    print(f"   • Fraud Rate: {fraud_rate:.3%}")
    
    if fraud_rate < 0.1:
        print(f"   ⚠️ HIGHLY IMBALANCED DATASET - Will need special handling!")

# Display first few rows
print("\n📋 TRAINING DATA PREVIEW:")
display(train_data.head())

# Data types and missing values
print("\n🕳️ MISSING VALUES ANALYSIS:")
missing_train = train_data.isnull().sum()
missing_train_pct = (missing_train / len(train_data)) * 100

missing_summary = pd.DataFrame({
    'Missing_Count': missing_train,
    'Missing_Percentage': missing_train_pct
})

missing_summary = missing_summary[missing_summary['Missing_Count'] > 0]
missing_summary = missing_summary.sort_values('Missing_Percentage', ascending=False)

if len(missing_summary) > 0:
    print("🚨 Columns with missing values:")
    display(missing_summary.round(2))
else:
    print("✅ No missing values found!")

## 2. Exploratory Data Analysis

In [ ]:
# Target variable distribution visualization
print("🎯 TARGET VARIABLE ANALYSIS")
print("=" * 50)

# Calculate fraud statistics
fraud_counts = train_data['isFraud'].value_counts()
fraud_percentages = train_data['isFraud'].value_counts(normalize=True) * 100

print(f"📊 Class Distribution:")
print(f"   • Legitimate (0): {fraud_counts[0]:,} transactions ({fraud_percentages[0]:.2f}%)")
print(f"   • Fraudulent (1): {fraud_counts[1]:,} transactions ({fraud_percentages[1]:.2f}%)")
print(f"   • Imbalance Ratio: 1:{fraud_counts[0]/fraud_counts[1]:.1f}")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Bar plot
fraud_counts.plot(kind='bar', ax=axes[0], color=['skyblue', 'salmon'])
axes[0].set_title('Fraud Distribution (Count)')
axes[0].set_xlabel('Is Fraud')
axes[0].set_ylabel('Number of Transactions')
axes[0].set_xticklabels(['Legitimate', 'Fraudulent'], rotation=0)

# Pie chart
axes[1].pie(fraud_counts.values, labels=['Legitimate', 'Fraudulent'], 
           autopct='%1.2f%%', colors=['skyblue', 'salmon'])
axes[1].set_title('Fraud Distribution (Percentage)')

# Log scale bar plot
fraud_counts.plot(kind='bar', ax=axes[2], color=['skyblue', 'salmon'], logy=True)
axes[2].set_title('Fraud Distribution (Log Scale)')
axes[2].set_xlabel('Is Fraud')
axes[2].set_ylabel('Number of Transactions (Log Scale)')
axes[2].set_xticklabels(['Legitimate', 'Fraudulent'], rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Transaction amount analysis
print("💳 TRANSACTION AMOUNT ANALYSIS")
print("=" * 50)

if 'TransactionAmt' in train_data.columns:
    fraud_amounts = train_data[train_data['isFraud'] == 1]['TransactionAmt']
    legit_amounts = train_data[train_data['isFraud'] == 0]['TransactionAmt']
    
    print(f"📈 Amount Statistics:")
    print(f"\nFraudulent Transactions:")
    print(f"   • Mean: ${fraud_amounts.mean():.2f}")
    print(f"   • Median: ${fraud_amounts.median():.2f}")
    print(f"   • Max: ${fraud_amounts.max():.2f}")
    
    print(f"\nLegitimate Transactions:")
    print(f"   • Mean: ${legit_amounts.mean():.2f}")
    print(f"   • Median: ${legit_amounts.median():.2f}")
    print(f"   • Max: ${legit_amounts.max():.2f}")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Box plot comparison
    train_data.boxplot(column='TransactionAmt', by='isFraud', ax=axes[0])
    axes[0].set_title('Transaction Amount by Fraud Status')
    axes[0].set_xlabel('Is Fraud')
    axes[0].set_ylabel('Transaction Amount ($)')
    
    # Histogram comparison
    axes[1].hist(legit_amounts, bins=50, alpha=0.7, label='Legitimate', density=True)
    axes[1].hist(fraud_amounts, bins=50, alpha=0.7, label='Fraudulent', density=True)
    axes[1].set_title('Transaction Amount Distribution')
    axes[1].set_xlabel('Transaction Amount ($)')
    axes[1].set_ylabel('Density')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Feature correlation analysis
print("🔗 FEATURE CORRELATION ANALYSIS")
print("=" * 50)

# Get numerical columns for correlation analysis
numerical_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
if 'TransactionID' in numerical_cols:
    numerical_cols.remove('TransactionID')

print(f"📊 Analyzing correlations for {len(numerical_cols)} numerical features")

if len(numerical_cols) > 1:
    # Calculate correlations with target
    correlations_with_target = train_data[numerical_cols].corrwith(train_data['isFraud']).sort_values(ascending=False)
    correlations_with_target = correlations_with_target.drop('isFraud', errors='ignore')
    
    print(f"\n🎯 Top correlations with fraud target:")
    print("\nTop 10 features by absolute correlation:")
    top_corr = correlations_with_target.abs().sort_values(ascending=False).head(10)
    for feature, corr in top_corr.items():
        actual_corr = correlations_with_target[feature]
        print(f"   • {feature}: {actual_corr:.4f}")
    
    # Visualization
    plt.figure(figsize=(12, 8))
    correlations_with_target.plot(kind='barh', color=['red' if x < 0 else 'blue' for x in correlations_with_target])
    plt.title('Feature Correlations with Fraud Target')
    plt.xlabel('Correlation Coefficient')
    plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

# Categorical features analysis
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()
print(f"\n📊 CATEGORICAL FEATURES ANALYSIS")
print(f"Categorical columns found: {len(categorical_cols)}")

if len(categorical_cols) > 0:
    for col in categorical_cols[:3]:  # Analyze first 3 categorical columns
        if col in train_data.columns:
            print(f"\n📊 Analysis for {col}:")
            cat_analysis = train_data.groupby(col, dropna=False)['isFraud'].agg(['count', 'sum', 'mean'])
            cat_analysis.columns = ['Total', 'Fraud_Count', 'Fraud_Rate']
            cat_analysis = cat_analysis.sort_values('Fraud_Rate', ascending=False)
            display(cat_analysis.head().round(4))

## 3. Data Preprocessing and Feature Engineering

In [ ]:
# Data preprocessing
print("🛠️ DATA PREPROCESSING")
print("=" * 50)

# Create a copy for preprocessing
df_processed = train_data.copy()

# Separate features and target
if 'TransactionID' in df_processed.columns:
    df_processed = df_processed.drop('TransactionID', axis=1)

X = df_processed.drop('isFraud', axis=1)
y = df_processed['isFraud']

print(f"📊 Features shape: {X.shape}")
print(f"🎯 Target shape: {y.shape}")

# Handle missing values
print(f"\n🔧 Handling missing values...")
missing_counts = X.isnull().sum()
columns_with_missing = missing_counts[missing_counts > 0].index.tolist()

if len(columns_with_missing) > 0:
    print(f"Found missing values in {len(columns_with_missing)} columns")
    
    for col in columns_with_missing:
        if X[col].dtype == 'object':
            # Fill categorical missing values with 'Missing'
            X[col] = X[col].fillna('Missing')
            print(f"   • {col}: Filled categorical missing values with 'Missing'")
        else:
            # Fill numerical missing values with median
            X[col] = X[col].fillna(X[col].median())
            print(f"   • {col}: Filled numerical missing values with median")
else:
    print("✅ No missing values found")

# Feature engineering
print(f"\n🔧 Feature engineering...")

# Create new features if TransactionAmt exists
if 'TransactionAmt' in X.columns:
    # Amount-based features
    X['TransactionAmt_log'] = np.log1p(X['TransactionAmt'])
    X['TransactionAmt_sqrt'] = np.sqrt(X['TransactionAmt'])
    X['IsHighAmount'] = (X['TransactionAmt'] > X['TransactionAmt'].quantile(0.95)).astype(int)
    print("   • Created amount-based features")

# Count missing values across features for each transaction
original_missing = train_data.drop(['isFraud', 'TransactionID'], axis=1, errors='ignore').isnull().sum(axis=1)
X['MissingValues_Count'] = original_missing
print("   • Created missing values count feature")

# Encode categorical variables
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
print(f"\n🏷️ Encoding {len(categorical_features)} categorical features...")

label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
    print(f"   • Encoded {col}")

print(f"\n✅ Preprocessing completed")
print(f"📊 Final feature set shape: {X.shape}")
print(f"🔍 Feature names: {list(X.columns[:10])}{'...' if len(X.columns) > 10 else ''}")

## 4. Train-Test Split and Class Imbalance Handling

In [ ]:
# Split the data
print("✂️ TRAIN-TEST SPLIT")
print("=" * 30)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Training set: {X_train.shape[0]} samples")
print(f"📊 Test set: {X_test.shape[0]} samples")
print(f"🎯 Training fraud rate: {y_train.mean():.3%}")
print(f"🎯 Test fraud rate: {y_test.mean():.3%}")

# Handle class imbalance
print(f"\n⚖️ HANDLING CLASS IMBALANCE")
print("=" * 40)

# Original distribution
original_fraud_count = y_train.sum()
original_legit_count = len(y_train) - original_fraud_count
print(f"Original distribution - Legitimate: {original_legit_count}, Fraudulent: {original_fraud_count}")

# Apply SMOTE if available, otherwise use simple oversampling
if IMBALANCE_AVAILABLE:
    print("🔄 Applying SMOTE oversampling...")
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
else:
    print("🔄 Applying random oversampling...")
    # Simple random oversampling
    fraud_indices = y_train[y_train == 1].index
    legit_indices = y_train[y_train == 0].index
    
    # Oversample fraud cases to balance
    n_fraud_needed = len(legit_indices) - len(fraud_indices)
    fraud_oversample_indices = np.random.choice(fraud_indices, size=n_fraud_needed, replace=True)
    
    # Combine original and oversampled indices
    balanced_indices = np.concatenate([legit_indices, fraud_indices, fraud_oversample_indices])
    
    X_train_balanced = X_train.loc[balanced_indices]
    y_train_balanced = y_train.loc[balanced_indices]

# Check new distribution
new_fraud_count = y_train_balanced.sum()
new_legit_count = len(y_train_balanced) - new_fraud_count
print(f"Balanced distribution - Legitimate: {new_legit_count}, Fraudulent: {new_fraud_count}")
print(f"New fraud rate: {y_train_balanced.mean():.3%}")
print(f"✅ Class imbalance handled successfully")

## 5. Model Implementation and Training

In [ ]:
# Define evaluation functions
def evaluate_fraud_model(y_true, y_pred, y_pred_proba, model_name):
    """
    Comprehensive evaluation for fraud detection models
    """
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred_proba)
    auc_pr = average_precision_score(y_true, y_pred_proba)
    
    print(f"\n📊 {model_name} Performance:")
    print(f"   • Accuracy: {accuracy:.4f}")
    print(f"   • Precision: {precision:.4f}")
    print(f"   • Recall: {recall:.4f}")
    print(f"   • F1-Score: {f1:.4f}")
    print(f"   • ROC AUC: {auc_roc:.4f}")
    print(f"   • PR AUC: {auc_pr:.4f}")
    
    return {
        'Accuracy': accuracy, 'Precision': precision, 'Recall': recall,
        'F1': f1, 'ROC_AUC': auc_roc, 'PR_AUC': auc_pr
    }

# Initialize fraud detection models
fraud_models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(random_state=42, max_iter=300)
}

print(f"🤖 TRAINING FRAUD DETECTION MODELS")
print("=" * 50)
print(f"Initialized {len(fraud_models)} models for fraud detection")

In [ ]:
# Train models with scaling pipeline
results = {}
trained_models = {}
scalers = {}

for name, model in fraud_models.items():
    print(f"\n🔄 Training {name}...")
    
    # Create pipeline with scaling
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    
    # Train the model
    pipeline.fit(X_train_balanced, y_train_balanced)
    
    # Make predictions
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # Evaluate
    metrics = evaluate_fraud_model(y_test, y_pred, y_pred_proba, name)
    results[name] = metrics
    trained_models[name] = pipeline

print("\n✅ All models trained successfully!")

In [ ]:
# Model comparison
print("📊 MODEL COMPARISON")
print("=" * 30)

results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('F1', ascending=False)

print("\n🏆 Model Rankings (by F1-Score):")
display(results_df.round(4))

# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC_AUC', 'PR_AUC']

for i, metric in enumerate(metrics):
    ax = axes[i//3, i%3]
    results_df[metric].plot(kind='bar', ax=ax, color='skyblue')
    ax.set_title(f'{metric} Comparison')
    ax.set_xlabel('Models')
    ax.set_ylabel(metric)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Best model
best_model_name = results_df.index[0]
best_f1 = results_df.loc[best_model_name, 'F1']
print(f"\n🥇 Best performing model: {best_model_name} (F1-Score: {best_f1:.4f})")

## 6. Detailed Model Analysis

In [ ]:
# Detailed analysis of best model
print(f"🔍 DETAILED ANALYSIS: {best_model_name}")
print("=" * 50)

best_model = trained_models[best_model_name]
y_pred_best = best_model.predict(X_test)
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_best)
print("\n📊 Confusion Matrix:")
print(cm)

# Classification Report
print("\n📋 Detailed Classification Report:")
print(classification_report(y_test, y_pred_best, target_names=['Legitimate', 'Fraudulent']))

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Confusion Matrix Heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
           xticklabels=['Legitimate', 'Fraudulent'],
           yticklabels=['Legitimate', 'Fraudulent'])
axes[0,0].set_title('Confusion Matrix')
axes[0,0].set_ylabel('True Label')
axes[0,0].set_xlabel('Predicted Label')

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_best)
axes[0,1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {results_df.loc[best_model_name, "ROC_AUC"]:.4f})')
axes[0,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[0,1].set_xlabel('False Positive Rate')
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].set_title('ROC Curve')
axes[0,1].legend()

# Precision-Recall Curve
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_proba_best)
axes[1,0].plot(recall_vals, precision_vals, color='blue', lw=2,
              label=f'PR curve (AUC = {results_df.loc[best_model_name, "PR_AUC"]:.4f})')
axes[1,0].set_xlabel('Recall')
axes[1,0].set_ylabel('Precision')
axes[1,0].set_title('Precision-Recall Curve')
axes[1,0].legend()

# Prediction Probability Distribution
fraud_proba = y_pred_proba_best[y_test == 1]
legit_proba = y_pred_proba_best[y_test == 0]

axes[1,1].hist(legit_proba, bins=30, alpha=0.7, label='Legitimate', density=True)
axes[1,1].hist(fraud_proba, bins=30, alpha=0.7, label='Fraudulent', density=True)
axes[1,1].set_xlabel('Predicted Fraud Probability')
axes[1,1].set_ylabel('Density')
axes[1,1].set_title('Prediction Probability Distribution')
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Business Impact Analysis
print(f"\n💰 BUSINESS IMPACT ANALYSIS")
print("=" * 40)

tn, fp, fn, tp = cm.ravel()
print(f"True Positives (Correctly detected fraud): {tp}")
print(f"False Positives (False alarms): {fp}")
print(f"False Negatives (Missed fraud): {fn}")
print(f"True Negatives (Correctly identified legitimate): {tn}")

if 'TransactionAmt' in X.columns:
    # Estimate financial impact (simplified)
    avg_fraud_amount = train_data[train_data['isFraud'] == 1]['TransactionAmt'].mean()
    detected_fraud_value = tp * avg_fraud_amount
    missed_fraud_value = fn * avg_fraud_amount
    
    print(f"\n💵 Estimated Financial Impact:")
    print(f"   • Fraud detected and prevented: ${detected_fraud_value:,.2f}")
    print(f"   • Fraud missed (potential loss): ${missed_fraud_value:,.2f}")
    print(f"   • Detection rate: {tp/(tp+fn):.1%}")

## 7. Feature Importance Analysis

In [ ]:
# Feature importance analysis
print(f"🔍 FEATURE IMPORTANCE ANALYSIS: {best_model_name}")
print("=" * 60)

try:
    model_step = best_model.named_steps['model']
    feature_names = X.columns
    
    if hasattr(model_step, 'feature_importances_'):
        # Tree-based models
        importance = model_step.feature_importances_
        
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
        print(f"\n🏆 Top 15 Most Important Features:")
        display(importance_df.head(15).round(4))
        
        # Visualization
        plt.figure(figsize=(12, 8))
        top_features = importance_df.head(15)
        plt.barh(range(len(top_features)), top_features['importance'])
        plt.yticks(range(len(top_features)), top_features['feature'])
        plt.xlabel('Feature Importance')
        plt.title(f'Top 15 Feature Importance - {best_model_name}')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        
    elif hasattr(model_step, 'coef_'):
        # Linear models
        coefficients = model_step.coef_[0]
        
        coef_df = pd.DataFrame({
            'feature': feature_names,
            'coefficient': coefficients,
            'abs_coefficient': np.abs(coefficients)
        }).sort_values('abs_coefficient', ascending=False)
        
        print(f"\n🏆 Top 15 Most Influential Features:")
        display(coef_df.head(15).round(4))
        
        # Visualization
        plt.figure(figsize=(12, 8))
        top_features = coef_df.head(15)
        colors = ['red' if x < 0 else 'blue' for x in top_features['coefficient']]
        plt.barh(range(len(top_features)), top_features['coefficient'], color=colors)
        plt.yticks(range(len(top_features)), top_features['feature'])
        plt.xlabel('Coefficient Value')
        plt.title(f'Top 15 Feature Coefficients - {best_model_name}')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
    
    else:
        print(f"Feature importance not available for {best_model_name}")
        
except Exception as e:
    print(f"Could not extract feature importance: {e}")

## 8. Final Results and Recommendations

In [ ]:
# Final comprehensive results
print("=" * 70)
print("           FRAUD DETECTION PIPELINE - FINAL RESULTS")
print("=" * 70)

print(f"\n📊 DATASET SUMMARY:")
print(f"   • Total transactions analyzed: {len(train_data):,}")
print(f"   • Fraudulent transactions: {train_data['isFraud'].sum():,} ({train_data['isFraud'].mean():.2%})")
print(f"   • Features used: {X.shape[1]}")
print(f"   • Test set size: {len(X_test):,} transactions")

print(f"\n🏆 BEST MODEL: {best_model_name}")
best_metrics = results_df.loc[best_model_name]
print(f"   • F1-Score: {best_metrics['F1']:.4f}")
print(f"   • Precision: {best_metrics['Precision']:.4f} (% of fraud predictions that are correct)")
print(f"   • Recall: {best_metrics['Recall']:.4f} (% of actual fraud detected)")
print(f"   • ROC AUC: {best_metrics['ROC_AUC']:.4f}")
print(f"   • PR AUC: {best_metrics['PR_AUC']:.4f}")

print(f"\n🎯 MODEL INTERPRETATION:")
if best_metrics['Precision'] > 0.8:
    print(f"   • HIGH PRECISION: Low false alarm rate - good for automated blocking")
elif best_metrics['Precision'] > 0.5:
    print(f"   • MODERATE PRECISION: Some false alarms - recommend manual review")
else:
    print(f"   • LOW PRECISION: High false alarm rate - needs improvement")

if best_metrics['Recall'] > 0.8:
    print(f"   • HIGH RECALL: Catches most fraud - good fraud detection coverage")
elif best_metrics['Recall'] > 0.5:
    print(f"   • MODERATE RECALL: Misses some fraud - acceptable for initial deployment")
else:
    print(f"   • LOW RECALL: Misses significant fraud - needs improvement")

print(f"\n📈 BUSINESS RECOMMENDATIONS:")
print(f"   1. Deploy {best_model_name} for real-time fraud scoring")
print(f"   2. Set fraud threshold based on business risk tolerance")
print(f"   3. Implement manual review process for borderline cases")
print(f"   4. Monitor model performance and retrain regularly")
print(f"   5. Focus on top {min(10, len(X.columns))} most important features for rule-based systems")

if 'TransactionAmt' in X.columns:
    avg_fraud_amount = train_data[train_data['isFraud'] == 1]['TransactionAmt'].mean()
    daily_transactions = 1000  # Assume 1000 transactions per day
    daily_fraud_rate = train_data['isFraud'].mean()
    daily_fraud_prevented = daily_transactions * daily_fraud_rate * best_metrics['Recall'] * avg_fraud_amount
    
    print(f"\n💰 ESTIMATED BUSINESS IMPACT (per day):")
    print(f"   • Potential fraud prevented: ${daily_fraud_prevented:,.2f}")
    print(f"   • Fraud detection rate: {best_metrics['Recall']:.1%}")
    print(f"   • False positive rate: {fp/(fp+tn):.1%}")

print(f"\n🔧 TECHNICAL IMPLEMENTATION:")
print(f"   • Model: {best_model_name} with StandardScaler preprocessing")
print(f"   • Features: {X.shape[1]} engineered features including amount transformations")
print(f"   • Class balancing: {'SMOTE' if IMBALANCE_AVAILABLE else 'Random oversampling'} applied")
print(f"   • Validation: Stratified train-test split with {len(X_test)} test samples")

print(f"\n✅ PIPELINE COMPLETION STATUS:")
print(f"   ✓ Data loaded and preprocessed")
print(f"   ✓ Exploratory analysis completed")
print(f"   ✓ Feature engineering applied")
print(f"   ✓ Class imbalance handled")
print(f"   ✓ {len(fraud_models)} models trained and evaluated")
print(f"   ✓ Best model selected and analyzed")
print(f"   ✓ Business impact quantified")
print(f"   ✓ Ready for production deployment")

## Summary and Next Steps

### 🎯 **Pipeline Achievements:**

1. **✅ Complete End-to-End Pipeline**: From raw data to production-ready fraud detection model
2. **✅ Comprehensive EDA**: Deep analysis of fraud patterns and risk factors  
3. **✅ Advanced Preprocessing**: Missing value handling, feature engineering, and encoding
4. **✅ Class Imbalance Solutions**: SMOTE/oversampling to handle typical fraud data challenges
5. **✅ Multiple Model Comparison**: Evaluated 7 different algorithms with proper metrics
6. **✅ Business Impact Analysis**: Quantified financial benefits and detection capabilities
7. **✅ Production Readiness**: Model interpretation and deployment recommendations

### 🚀 **For Production Deployment:**

1. **Save the best model** using joblib/pickle for serving
2. **Create API endpoint** for real-time fraud scoring
3. **Implement monitoring** for model drift and performance
4. **Set up retraining pipeline** for continuous learning
5. **Design alert system** for high-risk transactions

### 📈 **Model Improvement Opportunities:**

1. **Hyperparameter tuning** for the best performing model
2. **Ensemble methods** combining multiple models
3. **Deep learning approaches** for complex pattern recognition
4. **Time-based features** for transaction sequence analysis
5. **External data integration** (device fingerprinting, geolocation)

### 💡 **Key Learnings:**

- **Class imbalance** is critical in fraud detection and must be addressed
- **Feature engineering** significantly impacts model performance
- **Business metrics** (precision/recall trade-off) are more important than accuracy
- **Model interpretability** is crucial for fraud detection systems
- **Continuous monitoring** is essential for maintaining effectiveness